In [ ]:
#Cargar algo de drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Creacion de dataset y obtencion de datos

In [ ]:
from pandas.core.indexing import item_from_zerodim
import os,glob
import pandas as pd
import json,re

# Ruta de la carpeta principal
ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'
lista_part=["dev/parsed_pdfs/","test/parsed_pdfs/","train/parsed_pdfs/"]
lista_conf=["acl_2017"]
rutas_archivos_json = []
secc = {}

def limpiar_texto(texto):
    texto = re.sub(r'\n\d+\n', '\n', texto)
    texto = re.sub(r'[×/]', '', texto)
    texto = re.sub(r'\b\d+\b', '', texto)
    texto = re.sub(r'\n+', '\n', texto).strip()
    return texto

intro_pattern = re.compile(r'(?:\d*\.?\s*)?(introduction)', re.IGNORECASE)
conclusion_pattern = re.compile(r'(?:\d*\.?\s*)?(conclusion)', re.IGNORECASE)

for confe in lista_conf:
  #print(confe)
  for particion in lista_part:
    #print(particion)
    ruta=(ruta_carpeta_principal+"/"+confe+"/"+particion)
    #print("Explorando ruta:", ruta)
    archivos=glob.glob(ruta+"/*.json")
    #print(archivos)
    particion_sin_ruta = particion.strip('/').split('/')[0]
    #print(particion_sin_ruta)

    for archivo in archivos:
      with open(archivo, 'r') as f:
        contenido = json.load(f)
        #print(contenido.keys())
        nombre=contenido["name"]
        diccontent=contenido["metadata"]
        #print(diccontent)
        t = diccontent["title"]
        #print(t)
        e = diccontent["emails"]
        #print(e)
        autores=diccontent["authors"]
        #print(autores)
        dicsecciones=diccontent["sections"]

        referencias_archivo = diccontent["references"]

        abstract= diccontent["abstractText"]
        #print(dicsecciones)
        cadena=""
        introduccion=""
        conclusion=""
        contenido=""

        #print(dicsecciones)
        if dicsecciones!=None:
          for item in dicsecciones:
            if item['heading'] == None:
                cadena = ""
                text=""
            else:
                cadena = cadena +" "+ item['heading'] + "\n"
                cadena += " " #limpiar_texto(cadena)+" "

                # Verificar si el heading coincide con el patron de introduccion y concllu
                if re.search(intro_pattern, item['heading']):
                  introduccion = item['text'] #limpiar_texto(item['text'])
                elif re.search(conclusion_pattern, item['heading']):
                  conclusion = item['text'] #limpiar_texto(item['text'])
                else:
                  text= item['text']# limpiar_texto(item['text'])
                  contenido += f"**{item['heading']}**\n"+ text+ "\n"

            texto = item['text'] #re.sub(r'(\n\d+)+\n', '',item['text'])
            #texto = limpiar_texto(texto)
            cadena = cadena+" "+texto+"\n"
          rutas_archivos_json.append({
                      "Conferencia":confe,
                      "Particion":particion_sin_ruta,
                      "Archivo": nombre,
                      #"Title": t,
                      "Emails": e,
                      "Autores": autores,
                      "Texto Completo": f"{t} {cadena} Abstract {abstract}",
                      "Abstract": abstract,
                      "Introducción": introduccion,
                      "Contenido": contenido,
                      "Conclusion": conclusion,
                      "Referencias": referencias_archivo
                  })
        else:
          print("Error en el archivo: "+nombre)

          rutas_archivos_json.append({
                      "Conferencia":confe,
                      "Particion":particion_sin_ruta,
                      "Archivo": nombre,
                      "Title": t,
                      "Emails": e,
                      "Autores": autores,
                      "Texto Completo": "",
                      "Abstract": abstract,
                      "Introducción": "",
                      "Contenido": "",
                      "Conclusion": "",
                      "Referencias": referencias_archivo
                  })


df = pd.DataFrame(rutas_archivos_json)
df.to_csv(ruta_carpeta_principal+"/"+"acl_primer_ciclo.csv", sep=',', index=False, encoding='utf-8')
print("Datos guardados en CSV:", ruta_carpeta_principal+"/"+"acl_primer_ciclo.csv")



Datos guardados en CSV: /content/drive/MyDrive/PeerRead-master/data/acl_primer_ciclo.csv


In [ ]:
from pandas.core.indexing import item_from_zerodim
import os, glob
import pandas as pd
import json, re

# Ruta de la carpeta principal
ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'
lista_part = ["dev/parsed_pdfs/", "test/parsed_pdfs/", "train/parsed_pdfs/"]
lista_conf = ["acl_2017"]
rutas_archivos_json = []
secc = {}

# Contadores para las particiones
contador_dev = 0
contador_test = 0
contador_train = 0

def limpiar_texto(texto):
    texto = re.sub(r'\n\d+\n', '\n', texto)
    texto = re.sub(r'[×/]', '', texto)
    texto = re.sub(r'\b\d+\b', '', texto)
    texto = re.sub(r'\n+', '\n', texto).strip()
    return texto

intro_pattern = re.compile(r'(?:\d*\.?\s*)?(introduction)', re.IGNORECASE)
conclusion_pattern = re.compile(r'(?:\d*\.?\s*)?(conclusion)', re.IGNORECASE)

for confe in lista_conf:
    for particion in lista_part:
        ruta = (ruta_carpeta_principal + "/" + confe + "/" + particion)
        archivos = glob.glob(ruta + "/*.json")
        particion_sin_ruta = particion.strip('/').split('/')[0]

        for archivo in archivos:
            with open(archivo, 'r') as f:
                contenido = json.load(f)
                nombre = contenido["name"]
                #print(nombre)
                diccontent = contenido["metadata"]
                t = diccontent["title"]

                e = diccontent["emails"]
                autores = diccontent["authors"]
                dicsecciones = diccontent["sections"]
                referencias_archivo = diccontent["references"]
                abstract = diccontent["abstractText"]
                cadena = ""
                introduccion = ""
                conclusion = ""
                contenido = ""

                if dicsecciones is not None:
                    for item in dicsecciones:
                        if item['heading'] is None:
                            cadena = ""
                            text = ""
                        else:
                            cadena = cadena + " " + item['heading'] + "\n"
                            cadena += " "
                            if re.search(intro_pattern, item['heading']):
                                introduccion = item['text']
                            elif re.search(conclusion_pattern, item['heading']):
                                conclusion = item['text']
                            else:
                                text = item['text']
                                contenido += f"**{item['heading']}**\n" + text + "\n"

                        texto = item['text']
                        cadena = cadena + " " + texto + "\n"
                    rutas_archivos_json.append({
                        "Conferencia": confe,
                        "Particion": particion_sin_ruta,
                        "Archivo": nombre,
                        "Title": t,
                        "Emails": e,
                        "Autores": autores,
                        "Texto Completo": f"{t} {cadena} Abstract {abstract}",
                        "Abstract": abstract,
                        "Introducción": introduccion,
                        "Contenido": contenido,
                        "Conclusion": conclusion,
                        "Referencias": referencias_archivo
                    })

                    # Actualizar los contadores según la partición actual
                    if particion_sin_ruta == "dev":
                        contador_dev += 1
                    elif particion_sin_ruta == "test":
                        contador_test += 1
                    elif particion_sin_ruta == "train":
                        contador_train += 1
                else:
                    print("Error en el archivo: " + nombre)
                    rutas_archivos_json.append({
                        "Conferencia": confe,
                        "Particion": particion_sin_ruta,
                        "Archivo": nombre,
                        "Title": t,
                        "Emails": e,
                        "Autores": autores,
                        "Texto Completo": "",
                        "Abstract": abstract,
                        "Introducción": "",
                        "Contenido": "",
                        "Conclusion": "",
                        "Referencias": referencias_archivo
                    })

# Imprimir el número de archivos en cada partición
#print("Archivos en la partición 'dev':", contador_dev)
#print("Archivos en la partición 'test':", contador_test)
#print("Archivos en la partición 'train':", contador_train)

df = pd.DataFrame(rutas_archivos_json)
print(df)
df.to_csv(ruta_carpeta_principal + "/" + "acl_primer_ciclo.csv", sep=',', index=False, encoding='utf-8')
print("Datos guardados en CSV:", ruta_carpeta_principal + "/" + "acl_primer_ciclo.csv")


    Conferencia Particion  Archivo  \
0      acl_2017       dev  660.pdf   
1      acl_2017       dev  371.pdf   
2      acl_2017       dev  352.pdf   
3      acl_2017       dev  489.pdf   
4      acl_2017       dev   37.pdf   
..          ...       ...      ...   
132    acl_2017     train  419.pdf   
133    acl_2017     train  182.pdf   
134    acl_2017     train  699.pdf   
135    acl_2017     train  676.pdf   
136    acl_2017     train  691.pdf   

                                                                                                                              Title  \
0                                                                      Automatically Generating Rhythmic Verse with Neural Networks   
1                                                                                                                              None   
2                                                                                                                              None   
3  

# Obtener estado de aceptacion

In [ ]:
import pandas as pd

# Ruta del archivo
archivo_ruta = '/content/drive/MyDrive/PeerRead-master/data/acl_accepted.txt'

# Lista para almacenar los datos
data = []

# Leer el archivo
with open(archivo_ruta, 'r') as archivo:
    for linea in archivo:
        # Agregar una nueva fila con 'Title' y 'Accepted' como TRUE
        data.append({'Title': linea.strip(), 'Accepted': '1'})

# Crear un DataFrame de Pandas
df = pd.DataFrame(data)

# Eliminar filas duplicadas basadas en la columna 'Title'
df = df.drop_duplicates(subset='Title')

# Mostrar las primeras filas del DataFrame
print(df)

# Guardar el DataFrame en un archivo CSV
df.to_csv('/content/drive/MyDrive/PeerRead-master/data/reviews_acl.csv', index=False)


                                                                                                              Title  \
0                                               Token and Type Constraints for Cross-Lingual Part-of-Speech Tagging   
1                                                                         Finding Optimal 1-Endpoint-Crossing Trees   
2                                                                           Grounding Action Descriptions in Videos   
3                                         Branch and Bound Algorithm for Dependency Parsing with Non-local Features   
4                                Weakly Supervised Learning of Semantic Parsers for Mapping Instructions to Actions   
...                                                                                                             ...   
19738                                                       Cross-Lingual Syntactic Transfer with Limited Resources   
19739                                     Overco

# Combinar csv

In [ ]:
import os
import pandas as pd

# Especifica la ruta de tus archivos CSV en tu Google Drive
ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'

archivo1 = os.path.join(ruta_carpeta_principal, "acl_miguel.csv")
archivo2 = os.path.join(ruta_carpeta_principal, "reviews_acl.csv")

# Carga los archivos CSV en DataFrames
df1 = pd.read_csv(archivo1)
df2 = pd.read_csv(archivo2)

# Realiza la unión en base a la columna 'Title'
result = pd.merge(df1, df2, on='Title', how='outer')

# Llena las columnas 'Accepted' nulas con 'FALSE'
result['Accepted'] = result['Accepted'].fillna('0')
#result = result.dropna(subset=['Accepted'])
result = result.dropna(subset=['Archivo'])

print(result)
# Guarda el resultado en un nuevo archivo CSV
result.to_csv('/content/drive/MyDrive/PeerRead-master/data/acl_datos_miguel.csv', index=False)


     Archivo  \
0    660.pdf   
1    371.pdf   
2    352.pdf   
3     71.pdf   
4    343.pdf   
..       ...   
132  419.pdf   
133  182.pdf   
134  676.pdf   
135  691.pdf   
136  104.pdf   

                                                                                    Title  \
0                            Automatically Generating Rhythmic Verse with Neural Networks   
1                                                        Phrasal Recurrent Neural Network   
2                                 Adversarial Multi-task Learning for Text Classification   
3                                Cross-lingual Name Tagging and Linking for 282 Languages   
4                                          Neural Word Segmentation with Rich Pretraining   
..                                                                                    ...   
132                        One-Shot Neural Cross-Lingual Transfer for Paradigm Completion   
133  Modeling Contextual Relationships Among Utterances for Mul

In [ ]:
import os
import pandas as pd

# Especifica la ruta de tus archivos CSV en tu Google Drive
ruta_carpeta_principal = '/content/drive/MyDrive/PeerRead-master/data'

archivo1 = os.path.join(ruta_carpeta_principal, "acl_primer_ciclo.csv")
archivo2 = os.path.join(ruta_carpeta_principal, "acl_datos_miguel.csv")

# Carga los archivos CSV en DataFrames
df1 = pd.read_csv(archivo1)
df2 = pd.read_csv(archivo2)

# Guarda el número inicial de filas en cada DataFrame
num_filas_inicial_df1 = len(df1)
print("Filas del doc 1:",num_filas_inicial_df1)
num_filas_inicial_df2 = len(df2)
print("Filas del doc 2:",num_filas_inicial_df2)

# Union de csv con base en la columna 'Title'
result = pd.merge(df1, df2, on='Archivo', how='outer')
print("Número total de elementos al unir:", len(result))

# Calcular la diferencia en el número de filas después de la unión
#num_filas_eliminadas_despues_union = 0

result['Accepted'] = result['Accepted'].fillna('0')

# Dropear filas con valores nulos en columnas
#result = result.dropna(subset=['Accepted'])
#result = result.dropna(subset=['Contenido'])
print("Número total de elementos despues de dropear:", len(result))
# Calcular la diferencia en el numero de filas despues de la limpieza

#num_filas_eliminadas_despues_limpieza = 0  # No se eliminan filas en esta etapa ya que las líneas están comentadas
#print("Número total de filas eliminadas en limpieza:", num_filas_eliminadas_despues_limpieza)

#print(result)
# Guarda el resultado en un CSV
result.to_csv('/content/drive/MyDrive/acl_datos_completos.csv', index=False)


# Imprime el numero total de filas eliminadas
#print("Número total de filas eliminadas:", num_total_filas_eliminadas)



Filas del doc 1: 137
Filas del doc 2: 137
Número total de elementos al unir: 137
Número total de elementos despues de dropear: 137
